<a href="https://colab.research.google.com/github/Baldros/Neural-Networks/blob/main/RNN_MIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação:

Saber lidar com **dados sequênciais** é uma habilidade importantíssima para um cientista de dados. O objetivo desse código então é, baseado na aula do **MIT** da professora [Ava Amini](https://www.mit.edu/~asolei/), que trata de modelos de **Redes Neurais** para **séries temporais**, construir do zero um modelo **Recurrent Neural Networks (RNN's)** para tradução e geração de música.

## Material:
* [MIT 6.S191: Recurrent Neural Networks, Transformers, and Attention
](https://www.youtube.com/watch?v=dqoEU9Ac3ek)

* [Capítulo 10: Sequence Modeling: Recurrentand Recursive Nets (Goodfellow)](https://www.deeplearningbook.org/contents/rnn.html)



In [ ]:
# Biblioteca utilizada:
import tensorflow as tf

# Recurrent Neural Networks (RNN):

## Modelando Sequências: Critérios de projeto

Para modelas sequência, precisamos de:

1. lidar com sequências de comprimento variável;

2. Trackear dependencias de longo prazo;

3. Manter a informação em ordem;

4. Compartilhar os parâmetros para toda a sequência.

E uma **RNN** trata de todas essas questões.

## Intuição do Modelo:

A ideia básica é construir um modelo cujas as camadas ocultas (_hidden layers_) dependam dos estados ocultos das etapas anteriores. Ou seja, a saída $\hat{y}_t$
depende do estado oculto anterior $h_{t-1}$ (memoria passada) e do input atual $x_t$ (presente), sendo $x_t \in \mathbb{R}^m$ e $\hat{y}_t \in \mathbb{R}^n$, de modo que,

$$\hat{y}_t= f(x_t, h_{t-1})$$

**RNN's** possuem um estado $h_t$ que é atualizado a cada passo a medida que a sequência é processada. Apricando a **Relação de Recorrencia** para cada passo de de processamento da sequência, temos:

$$h_t = f_W(x_t, h_{t-1})$$

Onde:

* $h_t$: É o estado da célula;;
* $f_w$: É a função com os pesso $W$;
* $x_t$: vetor de entradas;
* $h_{t-1}$: Estado anterior da célula.

É importante dizer que a mesma função e conjunto de parâmetros são usados para cada passo do aprendizado.

In [ ]:
# Função de Predição:
def predict(sentence, model):
  '''
  Função que realiza a predição do modelo.
  '''

  # Estruturando as entradas e as hidden layers:
  list_of_words = sentence.split() # Splitando as palavras
  hidden_state = [0]*len(list_of_words)

  # Feedforward:
  for word in list_of_words:
    prediction, hidden_state = model(word,hidden_state)

  return [prediction, hidden_state]

Esse é o core do processo, vamos agroa construir o modelo etapa por etapa


## Atualização dos estados e saídas:

Vimos que a ideia aqui é trabalhar com camadas ocultas dependentes, tal que $\hat{y}_t= f(x_t, h_{t-1})$. Refinando então essa ideia, temos que,

$$h_t =\text{tanh}(W_{hh}^Th_{t-1}+W_{xh}^Tx_t)$$

Deste modo então podemos definir a nossa saída, que se faz de forma que,

$$\hat{y}_t = W_{hy}^Th_t$$

## Computando a perda:

Essa Rede é baseada em grafos e como ja dito, a ideia aqui é reutilizar a mesma matriz de pesos para cada passo do aprendizado.

A ideia aqui é computar a perda para cada etapa do treinamento e somar, obtendo a perda total do modelo.

In [ ]:
class MyRNNCell(tf.keras.layers.Layer):
  def __init__(self, rnn_units,input_dim, output_dim):
    super(MyRNNCell, self).__init__()

    # Inicializando Matrizes de peso:
    self.W_xh = self.add_weight([rnn_units, input_dim])
    self.W_hh = self.add_weight([rnn_units, rnn_units])
    self.W_hy = self.add_weight([output_dim, rnn_units])

    # Instanciando estados ocultos:
    self.h = tf.zeros([rnn_units,1])

  def forwardpass(self, x):
    # Atualização das camadas ocultas:
    self.h = tf.math.tanh(self.W_hh*self.h + self.W_xh*x)

    # Computando a saída:
    output = self.W_hy*self.h

    return output

# Modelando Sequências: